In [1]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import laspy as lp
import pdal
import pandas as pd
import open3d as o3d
from dbfread import DBF

# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt

In [2]:
name = 'kiwifruit'

In [3]:
# load files
las = loads.loadlaz(name)
traj = loads.loaddbf(name)

In [26]:
traj = loads.loaddbf(name)
# get coordinates numpy array
# lidar
points = np.vstack((las.x, las.y, las.z)).transpose()

# trajectory
traj = pd.DataFrame(iter(traj)) # to pandas
points_t = np.vstack((traj.x, traj.y, traj.z)).transpose()

# plot the two point clouds
pointslist = [points[::10], points_t]
colours = [[0.5, 0.5, 0.5], [1, 0, 0]]
loads.showPCDS(pointslist, colours)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
# interpolate trajectory with gps time

df = loads.coordsDF(las, traj)

In [18]:
# show sample of beams
loads.showbeams(df[15000000:15000300])

In [134]:
df1 = df[::100]

plt.figure(figsize=(16, 8))
x = np.linspace(-60, 10, 20)
y = np.linspace(-30, 10, 20)
keep = (df1['y'] > 0.34*df1['x'] - 4.0) & (df1['y'] < 0.34*df1['x'] + 6)
keep &= (df1['x'] < -0.35*df1['y'] - 10) & (df1['x'] > -0.35*df1['y'] - 60)
plt.scatter(df1['x'], df1['y'], s=1, c='gray', alpha=0.3)
plt.scatter(df1['x'][keep], df1['y'][keep], s=1)
plt.plot(x, 0.34*x - 4.0, lw=2, c='r')
plt.plot(x, 0.34*x + 6, lw=2, c='r')
plt.plot(-0.35*y - 10, y, lw=2, c='r')
plt.plot(-0.35*y - 60, y, lw=2, c='r')

plt.grid()

plt.xlim(-70, 20)
plt.ylim(-35,15)

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$y$', size=20)


plt.show()

## Get the foliage

In [170]:
# select the field
keep = (df['y'] > 0.34*df['x'] - 4.0) & (df['y'] < 0.34*df['x'] + 6)
keep &= (df['x'] < -0.35*df['y'] - 10) & (df['x'] > -0.35*df['y'] - 60)

# Set the foliage min and max height
keepz = (df['z'] > 1.2) & (df['z'] < 3.8)

# remove outliers using percentiles
res, mask = loads.remove_outliers(df['x'][keep & keepz], df['z'][keep & keepz])
leaves = np.zeros(len(df['x']), dtype=bool)
leaves[(keep) & (keepz)] = mask

In [138]:
df1 = df[keep][::1]
print(len(df))
print(len(df1))

plt.figure(figsize=(16, 8))
plt.scatter(df1['x'], df1['z'], s=0.01)
plt.axhline(1.2, lw=2, c='k')
plt.axhline(3.8, lw=2, c='k')

res, _ = loads.remove_outliers(df['x'][keep & keepz][::5], df['z'][keep & keepz][::5])

plt.grid()

plt.ylim(-2,10)

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$z$', size=20)

39341422
5732229


Text(0, 0.5, '$z$')

## Tree and leaves segmentation

In [206]:
def segtree(df, leaves, show=False):

    trees = {}
    bins = np.arange(10, 70, 10)

    if show:
        plt.figure(figsize=(16, 8))

    for i in range(len(bins)-1):
        keep = np.ones(len(df['x']), dtype=bool)
        keep &= (df['y'] > 0.34*df['x'] - 4.0) & (df['y'] < 0.34*df['x'] + 6)
        keep &= (df['x'] < -0.35*df['y'] - bins[i]) & (df['x'] > -0.35*df['y'] - bins[i+1])

        trees['tree_%s' %(str(i))] = keep
            
        if show:
            plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.01, label=i)
            box = dict(facecolor='green', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.4)
            text = 'tree_%s' %(str(i))
            xx = - (bins[i+1] + bins[i]) / 2
            plt.text(xx - i*xx*0.02, 0.34*xx + 1, text, size=20, bbox=box)

    if show:
        plt.xlabel(r'$x$', size=20)
        plt.ylabel(r'$y$', size=20)
        plt.show()

    return trees


In [207]:
trees = segtree(df, leaves, show=True)

In [268]:
# show the point cloud from leaves of firs tree only
keep = (trees['tree_0']) & (leaves)
loads.showPCfromDF(df[keep])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# `Leaf Inclination Angle` (LIA) estimation

On the contrary to the mock example, here we can not use function `lia.bestfit_pars_la` to get the best-fit parameters...

In [209]:
# load bestfit results
for key, val in trees.items():

    print('********* %s *********' %(key))

    keep = (val) & (leaves)
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df_)

    voxel_size_w = 0.01
    kd3_sr = 0.1
    max_nn = 15

    text = '%s=%.4f \n %s=%.4f \n %s=%.4f ' %(
                            'voxel_size_w', voxel_size_w,
                            'kd3_sr', kd3_sr,
                            'max_nn', max_nn)
    print(text)

    lia.leaf_angle(points, name, key, voxel_size_w, kd3_sr, max_nn, save=True,
                                savefig=True, text=text, voxel_size_h=0.1, ismock=False,
                                ylim=0.017, ylimh=0.35)

voxel_size_w=0.0100 
 kd3_sr=0.1000 
 max_nn=15.0000 
********* tree_0 *********
voxel_size_w=0.0100 
 kd3_sr=0.1000 
 max_nn=15.0000 
********* tree_1 *********
voxel_size_w=0.0100 
 kd3_sr=0.1000 
 max_nn=15.0000 
********* tree_2 *********
voxel_size_w=0.0100 
 kd3_sr=0.1000 
 max_nn=15.0000 
********* tree_3 *********
voxel_size_w=0.0100 
 kd3_sr=0.1000 
 max_nn=15.0000 
********* tree_4 *********


# `Leaf Area Density` (LAD) estimation

In [212]:
for key, val in trees.items():

    keep = (val)
    print(key, np.sum(keep))

tree_0 1412368
tree_1 1173406
tree_2 1088034
tree_3 1090216
tree_4 968205


In [277]:
downsample = 0.005
voxel_size = 0.15
# to check everything looks fine
show = False
sample = None

In [267]:

POINTS = loads.DF2array(df[['x', 'y', 'z']])
SENSORS = loads.DF2array(df[['xs', 'ys', 'zs']])

if downsample is not None:

    resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        inds = np.load(outdir)

        points = POINTS[inds]
        sensors = SENSORS[inds]

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, len(df), int(len(df) * downsample))
        inds = np.zeros(len(df), dtype=bool)
        inds[idx] = True

        points = POINTS[inds]
        sensors = SENSORS[inds]

        np.save(outdir, inds)

else:

    resdir = os.path.join(_data, name, 'lad')
    if not os.path.exists(resdir):
        os.makedirs(resdir)

if sample is not None:

    idx = np.random.randint(0, len(df), int(sample))
    points = POINTS[idx]
    sensors = SENSORS[idx]

for key, val in trees.items():

    inPR = (val) & (leaves) & (inds)
    pointsPR = POINTS[inPR]
    m3s, m3count= rayt.main(points, sensors, pointsPR, voxel_size, resdir, key, show=show)
    

inds file already exists for donwnsample of 0.005 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.005/inds.npy
max --> [80, 79, 13]
min --> [0, 0, 0]


196188it [11:33, 282.88it/s]


tot vox: 	 90720
voxels hitted: 	 84374
Percentage of voxels hitted by beam: 0.93
max --> [80, 78, 12]
min --> [0, 0, 0]


196188it [05:46, 566.50it/s] 


tot vox: 	 83187
voxels hitted: 	 70448
Percentage of voxels hitted by beam: 0.85
max --> [78, 79, 11]
min --> [0, 0, 0]


...

tot vox: 	 75840
voxels hitted: 	 62279
Percentage of voxels hitted by beam: 0.82
max --> [79, 78, 8]
min --> [0, 0, 0]


196188it [03:44, 873.75it/s]  


tot vox: 	 56880
voxels hitted: 	 45474
Percentage of voxels hitted by beam: 0.80
max --> [78, 78, 10]
min --> [0, 0, 0]


196188it [05:44, 568.91it/s]  

tot vox: 	 68651
voxels hitted: 	 54022
Percentage of voxels hitted by beam: 0.79


In [278]:
if downsample is not None:
    inds_file = os.path.join(resdir, 'inds.npy')
    inds = np.load(inds_file)
    resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
    print('downsample:', downsample)
else:
    inds = np.ones(len(df), dtype=bool)
    resdir = os.path.join(_data, name, 'lad')

isfigures = os.path.join(resdir, 'figures')
if not os.path.exists(isfigures):
    os.makedirs(isfigures)

print('voxel_size:', voxel_size)

for key, val in trees.items():

    # if key != 'tree_0': continue

    inPR = (val) & (leaves) & (inds)
    pointsPR = POINTS[inPR]
    sensorsPR = SENSORS[inPR]

    m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)
    # get in down sample boolean array for LPC size
    inds_ = inds[(val) & (leaves)]
    lias, ws = lad.downsample_lia(name, key, inds_)
    voxk = lad.get_voxk(pointsPR, voxel_size)
    bia = lad.get_bia(pointsPR, sensorsPR)
    # meshfile = lad.get_meshfile(name)

    figext = '%s_%s' %(key, str(voxel_size))
    # figext = None
    alphas_k = lad.alpha_k(bia, voxk, lias, ws, resdir, figext=figext, 
                            klia=False, use_true_lia=False)

    # kmax = m3att.shape[2]
    # kbins = int(kmax/10)
    # print(kbins)
    kbins = 1
    
    # lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
    # lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
    lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1)
    lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0)
    # lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

    lads = {'Correction Mean':lads_mid, 'No Correction':lads_0}

    savefig = os.path.join(resdir, 'figures','LAD_%s.png' %(figext))
    figures.plot_lads(lads, savefig=savefig)



downsample: 0.005
voxel_size: 0.15
max --> [80, 79, 13]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (81, 80, 14)
ray tracker voxel dimensions: 	 (81, 80, 14)
Number of voxels ocupied by points cloud: 	 4360
Number of voxels ocupied by beam points cloud: 	 84374
Total number of voxels in plant regions: 	 90720
Number of voxels with attribute 1: 	 4360
Number of voxels with attribute 2: 	 80014
Number of voxels with attribute 3: 	 6346
max --> [80, 78, 12]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (81, 79, 13)
ray tracker voxel dimensions: 	 (81, 79, 13)
Number of voxels ocupied by points cloud: 	 3773
Number of voxels ocupied by beam points cloud: 	 70448
Total number of voxels in plant regions: 	 83187
Number of voxels with attribute 1: 	 3773
Number of voxels with attribute 2: 	 66675
Number of voxels with attribute 3: 	 12739
max --> [78, 79, 11]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (79, 80, 12)
ray tracker voxel dimensions: 	 (79, 80, 12)
Number of voxels ocupied by p